# Users Web Scrapping

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time 
import csv 
from random import randint

In [4]:
def get_value(text):
    # Función para conseguir el texto entre los markups
    text = str(text)
    text = text.replace("\n", "").replace("\r", "").replace("\t", "")
    markup_count = text.count("</")
    position = -1
    for i in range(markup_count):
        position = text.find(">", position + 1)
    first_close = text.find("</")
    return text[position + 1: first_close]
    
def get_rating_value(text):
    # Función para conseguir el rating con el que se califica un anime
    text = str(text)
    text = text.replace("\n", "").replace("\r", "").replace("\t", "").replace(" ", "")
    rating_text = text.split(':')[-1]
    return rating_text.split('<')[0]

def get_anime_rating(text):
    # Función para conseguir un anime y su rating desde la lista de anime list de un usuario
    splited_obj = text.split(',')
    if len(splited_obj) == 28:
        anime_id = splited_obj[7].split('"anime_id":')[1]
        anime = splited_obj[4].split('"anime_title":"')[1][:-1]
        rating = splited_obj[0].split(':')[1]
        return (anime_id, anime, rating)
    return (None, None, None)
    
    
    

In [15]:
def product_female_user_list_data(total_data = 50000):
    '''
    Scrapping para obtener la calificación de los usuarios a los animé que han visto.
    Recibe como parámetro total_data, que corresponde al total de filas que se quiere obtener.
    '''

    base_url = 'https://myanimelist.net/'
    url = 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2'
    actual_url = 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2&show=5832'
    users_information = [[], []] # [ [username], [user gender]]
    ratings_information = [[], [], []] # [ [username], [anime], [overall rating] ]
    
    while True:
        print('actual url: ', actual_url)
        main_page = requests.get(actual_url)
        soup = BeautifulSoup(main_page.content, 'html.parser')
        tables = soup.find_all('table')
        user_rows = tables[1].find_all('tr')
    
        for row in user_rows:
            user_icons = row.find_all('td')
            for icon in user_icons:
                
                user_name = icon.find('div').find('a')
                user_name = get_value(user_name)
                users_information[0].append(user_name)
                
                user_href = icon.find('div').find('a', href=True)
                user_url = base_url + user_href['href']
                
                # Ingresamos al perfil del usuario para obtener género y anime list
                time.sleep(randint(1,6))
                user_page = requests.get(user_url)
                user_soup = BeautifulSoup(user_page.content, 'html.parser')
                user_info = user_soup.find('ul', {'class': 'user-status'})
                if user_info is None:
                    gender = 'Not Found'
                else:
                    span = user_info.find_all('li')[1].find_all('span')
                    if get_value(span[0]) == 'Gender':
                        gender = get_value(span[1])
                    else:
                        gender = 'None'
                users_information[1].append(gender)
                
                # Escribimos los datos del usuario en el csv
                with open('female_users.csv', mode='a') as users_file:
                    users_writer = csv.writer(users_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    users_writer.writerow([user_name, gender]) 
    
                user_lists = user_soup.find('div', {'class': 'user-button clearfix mb12'})
                user_anime_list = user_lists.find('a', href=True)
                user_anime_list_url = user_anime_list['href']
                
                # Ingresamos al anime list del usuario
                anime_list = requests.get(user_anime_list_url)
                anime_list_soup = BeautifulSoup(anime_list.content, 'html.parser')
                status_menu = anime_list_soup.find('div', {'class': 'status-menu-container'})
                if status_menu:
                    try:
                        completed_anime_list_url = status_menu.find('a', {'class': 'status-button completed'}, href=True)['href']

                        # Ingresamos a anime list completos para extraer su calificacion
                        completed_anime_list = requests.get(completed_anime_list_url)
                        completed_anime_list_soup = BeautifulSoup(completed_anime_list.content, 'html.parser')
                        completed_list = completed_anime_list_soup.find('div', {'class': 'list-unit completed'})

                        objects_completed_list = completed_list.find('table')['data-items']
                        objects_completed_list = objects_completed_list.split('"status":2,')

                        # Tomamos los datos de los distintos anime y guardamos el rating
                        for _object in objects_completed_list:
                            anime_id, anime, rating = get_anime_rating(_object)
                            if anime and rating:
                                ratings_information[0].append(user_name)
                                ratings_information[1].append(anime)
                                ratings_information[2].append(rating)
                                with open('female_ratings.csv', mode='a') as ratings_file:
                                    ratings_writer = csv.writer(ratings_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    ratings_writer.writerow([user_name, anime_id, anime, rating]) 

                        # Revisamos si conseguimos el total de usuarios que queremos
                        print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
                        if len(users_information[0]) == total_data:
                            print("break")
                            break
                            
                    except:
                        pass
            
            # Revisamos si conseguimos el total de usuarios que queremos
            print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
            if len(users_information[0]) == total_data:
                print("break")
                break
                
         # Revisamos si conseguimos el total de usuarios que queremos
        print("users:info[0]: ", len(users_information[0]), " total_data: ", total_data)
        if len(users_information[0]) == total_data:
            print("break")
            break
        
        #Si aún no hemos conseguido la cantidad de datos buscada, cambiamos de url a la siguiente página y realizamos nuevamente el proceso.
        if actual_url == 'https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2': 
            next_page = url + '&show=24'
        else:
            next_page = '&show=' + str(int(actual_url.split('=')[-1]) + 24)
        actual_url = url + next_page
  
        
    return 
     

In [16]:
product_female_user_list_data()

actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2&show=5832
users:info[0]:  1  total_data:  50000
users:info[0]:  2  total_data:  50000
users:info[0]:  3  total_data:  50000
users:info[0]:  4  total_data:  50000
users:info[0]:  5  total_data:  50000
users:info[0]:  6  total_data:  50000
users:info[0]:  6  total_data:  50000
users:info[0]:  7  total_data:  50000
users:info[0]:  8  total_data:  50000
users:info[0]:  9  total_data:  50000
users:info[0]:  10  total_data:  50000
users:info[0]:  11  total_data:  50000
users:info[0]:  12  total_data:  50000
users:info[0]:  12  total_data:  50000
users:info[0]:  13  total_data:  50000
users:info[0]:  14  total_data:  50000
users:info[0]:  15  total_data:  50000
users:info[0]:  16  total_data:  50000
users:info[0]:  17  total_data:  50000
users:info[0]:  18  total_data:  50000
users:info[0]:  18  total_data:  50000
users:info[0]:  19  total_data:  50000
users:info[0]:  20  total_data:  50000
users:info[0]:  

users:info[0]:  171  total_data:  50000
users:info[0]:  172  total_data:  50000
users:info[0]:  173  total_data:  50000
users:info[0]:  174  total_data:  50000
users:info[0]:  174  total_data:  50000
users:info[0]:  175  total_data:  50000
users:info[0]:  176  total_data:  50000
users:info[0]:  177  total_data:  50000
users:info[0]:  179  total_data:  50000
users:info[0]:  180  total_data:  50000
users:info[0]:  180  total_data:  50000
users:info[0]:  181  total_data:  50000
users:info[0]:  182  total_data:  50000
users:info[0]:  183  total_data:  50000
users:info[0]:  184  total_data:  50000
users:info[0]:  185  total_data:  50000
users:info[0]:  186  total_data:  50000
users:info[0]:  186  total_data:  50000
users:info[0]:  187  total_data:  50000
users:info[0]:  188  total_data:  50000
users:info[0]:  189  total_data:  50000
users:info[0]:  190  total_data:  50000
users:info[0]:  191  total_data:  50000
users:info[0]:  192  total_data:  50000
users:info[0]:  192  total_data:  50000


users:info[0]:  337  total_data:  50000
users:info[0]:  338  total_data:  50000
users:info[0]:  339  total_data:  50000
users:info[0]:  340  total_data:  50000
users:info[0]:  341  total_data:  50000
users:info[0]:  342  total_data:  50000
users:info[0]:  342  total_data:  50000
users:info[0]:  343  total_data:  50000
users:info[0]:  344  total_data:  50000
users:info[0]:  345  total_data:  50000
users:info[0]:  346  total_data:  50000
users:info[0]:  347  total_data:  50000
users:info[0]:  348  total_data:  50000
users:info[0]:  348  total_data:  50000
users:info[0]:  349  total_data:  50000
users:info[0]:  351  total_data:  50000
users:info[0]:  352  total_data:  50000
users:info[0]:  353  total_data:  50000
users:info[0]:  354  total_data:  50000
users:info[0]:  354  total_data:  50000
users:info[0]:  355  total_data:  50000
users:info[0]:  357  total_data:  50000
users:info[0]:  358  total_data:  50000
users:info[0]:  359  total_data:  50000
users:info[0]:  360  total_data:  50000


users:info[0]:  506  total_data:  50000
users:info[0]:  507  total_data:  50000
users:info[0]:  508  total_data:  50000
users:info[0]:  509  total_data:  50000
users:info[0]:  510  total_data:  50000
users:info[0]:  510  total_data:  50000
users:info[0]:  511  total_data:  50000
users:info[0]:  512  total_data:  50000
users:info[0]:  513  total_data:  50000
users:info[0]:  514  total_data:  50000
users:info[0]:  516  total_data:  50000
users:info[0]:  516  total_data:  50000
users:info[0]:  517  total_data:  50000
users:info[0]:  519  total_data:  50000
users:info[0]:  520  total_data:  50000
users:info[0]:  521  total_data:  50000
users:info[0]:  522  total_data:  50000
users:info[0]:  522  total_data:  50000
users:info[0]:  523  total_data:  50000
users:info[0]:  524  total_data:  50000
users:info[0]:  525  total_data:  50000
users:info[0]:  526  total_data:  50000
users:info[0]:  527  total_data:  50000
users:info[0]:  528  total_data:  50000
users:info[0]:  528  total_data:  50000


users:info[0]:  673  total_data:  50000
users:info[0]:  674  total_data:  50000
users:info[0]:  675  total_data:  50000
users:info[0]:  676  total_data:  50000
users:info[0]:  677  total_data:  50000
users:info[0]:  678  total_data:  50000
users:info[0]:  678  total_data:  50000
users:info[0]:  680  total_data:  50000
users:info[0]:  681  total_data:  50000
users:info[0]:  682  total_data:  50000
users:info[0]:  683  total_data:  50000
users:info[0]:  684  total_data:  50000
users:info[0]:  684  total_data:  50000
users:info[0]:  685  total_data:  50000
users:info[0]:  686  total_data:  50000
users:info[0]:  687  total_data:  50000
users:info[0]:  688  total_data:  50000
users:info[0]:  689  total_data:  50000
users:info[0]:  690  total_data:  50000
users:info[0]:  690  total_data:  50000
users:info[0]:  691  total_data:  50000
users:info[0]:  693  total_data:  50000
users:info[0]:  695  total_data:  50000
users:info[0]:  696  total_data:  50000
users:info[0]:  696  total_data:  50000


...

users:info[0]:  4288  total_data:  50000
users:info[0]:  4289  total_data:  50000
users:info[0]:  4290  total_data:  50000
users:info[0]:  4290  total_data:  50000
users:info[0]:  4291  total_data:  50000
users:info[0]:  4292  total_data:  50000
users:info[0]:  4293  total_data:  50000
users:info[0]:  4295  total_data:  50000
users:info[0]:  4296  total_data:  50000
users:info[0]:  4296  total_data:  50000
users:info[0]:  4296  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2&show=10128
users:info[0]:  4297  total_data:  50000
users:info[0]:  4298  total_data:  50000
users:info[0]:  4299  total_data:  50000
users:info[0]:  4300  total_data:  50000
users:info[0]:  4301  total_data:  50000
users:info[0]:  4302  total_data:  50000
users:info[0]:  4302  total_data:  50000
users:info[0]:  4303  total_data:  50000
users:info[0]:  4304  total_data:  50000
users:info[0]:  4305  total_data:  50000
users:info[0]:  4306  total_data:  50000


users:info[0]:  4449  total_data:  50000
users:info[0]:  4450  total_data:  50000
users:info[0]:  4451  total_data:  50000
users:info[0]:  4452  total_data:  50000
users:info[0]:  4452  total_data:  50000
users:info[0]:  4453  total_data:  50000
users:info[0]:  4454  total_data:  50000
users:info[0]:  4456  total_data:  50000
users:info[0]:  4457  total_data:  50000
users:info[0]:  4458  total_data:  50000
users:info[0]:  4458  total_data:  50000
users:info[0]:  4459  total_data:  50000
users:info[0]:  4460  total_data:  50000
users:info[0]:  4462  total_data:  50000
users:info[0]:  4463  total_data:  50000
users:info[0]:  4464  total_data:  50000
users:info[0]:  4464  total_data:  50000
users:info[0]:  4464  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2&show=10296
users:info[0]:  4465  total_data:  50000
users:info[0]:  4466  total_data:  50000
users:info[0]:  4467  total_data:  50000
users:info[0]:  4468  total_data:  50000


users:info[0]:  4611  total_data:  50000
users:info[0]:  4613  total_data:  50000
users:info[0]:  4614  total_data:  50000
users:info[0]:  4614  total_data:  50000
users:info[0]:  4615  total_data:  50000
users:info[0]:  4616  total_data:  50000
users:info[0]:  4617  total_data:  50000
users:info[0]:  4618  total_data:  50000
users:info[0]:  4619  total_data:  50000
users:info[0]:  4620  total_data:  50000
users:info[0]:  4620  total_data:  50000
users:info[0]:  4621  total_data:  50000
users:info[0]:  4622  total_data:  50000
users:info[0]:  4623  total_data:  50000
users:info[0]:  4624  total_data:  50000
users:info[0]:  4625  total_data:  50000
users:info[0]:  4626  total_data:  50000
users:info[0]:  4626  total_data:  50000
users:info[0]:  4628  total_data:  50000
users:info[0]:  4629  total_data:  50000
users:info[0]:  4630  total_data:  50000
users:info[0]:  4631  total_data:  50000
users:info[0]:  4632  total_data:  50000
users:info[0]:  4632  total_data:  50000
users:info[0]:  

users:info[0]:  4774  total_data:  50000
users:info[0]:  4775  total_data:  50000
users:info[0]:  4776  total_data:  50000
users:info[0]:  4776  total_data:  50000
users:info[0]:  4776  total_data:  50000
actual url:  https://myanimelist.net/users.php?cat=user&q=&loc=&agelow=0&agehigh=0&g=2&show=10608
users:info[0]:  4777  total_data:  50000
users:info[0]:  4778  total_data:  50000
users:info[0]:  4779  total_data:  50000
users:info[0]:  4780  total_data:  50000
users:info[0]:  4782  total_data:  50000
users:info[0]:  4782  total_data:  50000
users:info[0]:  4784  total_data:  50000
users:info[0]:  4785  total_data:  50000
users:info[0]:  4787  total_data:  50000
users:info[0]:  4788  total_data:  50000
users:info[0]:  4788  total_data:  50000
users:info[0]:  4790  total_data:  50000
users:info[0]:  4791  total_data:  50000
users:info[0]:  4792  total_data:  50000
users:info[0]:  4793  total_data:  50000
users:info[0]:  4794  total_data:  50000
users:info[0]:  4794  total_data:  50000


KeyboardInterrupt: 